In [16]:
import ipywidgets as widgets  # control library (computing)
from IPython.display import display, Markdown, clear_output  # Methods for displaying controls
from functools import partial
!pip install brian2          # Import jupyter directly and run it without this step.
from brian2 import *
import re

%matplotlib inline


def match_digital(Str):
    if type(Str) == Quantity:
        return Str
    matches = re.findall(r"(\d+)(\D+)", Str)
    if matches:
        value = float(matches[0][0])
        unit = matches[0][1]
        return value*eval(unit)
    else:
        print("Invalid format")
def match_digital2(Str):
    matches = re.findall(r"(\d+)(\D+)", Str)
    if matches:
        value = float(matches[0][0])
        unit = matches[0][1]
        return value,str2quantity(unit)
    else:
        print("Invalid format")

def create_gui(parameters):
    parameter_values = {}
    for param in parameters:
        parameter_values[param] = widgets.Text(description=param, value='None')
    accordion = widgets.Accordion()
    accordion.children = []
    accordion.children += (widgets.VBox(list(parameter_values.values())),)
    accordion.set_title(0, '参数')
    gui, parameter_values = accordion, parameter_values
    return gui, parameter_values


def str2quantity(unit_string):
    if unit_string == 'amp' or unit_string == 'ampere':
        return 1*amp
    elif unit_string == 'kilogram' or unit_string == 'kilogramme':
        return 1*kilogram
    elif unit_string == 'second':
        return 1*second
    elif unit_string == 'metre' or unit_string == 'meter':
        return 1*metre
    elif unit_string == 'mole' or unit_string == 'mol':
        return 1*mole
    elif unit_string == 'kelvin':
        return 1*kelvin
    elif unit_string == 'candela':
        return 1*candela
    elif unit_string == 'coulomb':
        return 1*coulomb
    elif unit_string == 'farad':
        return 1*farad
    elif unit_string == 'gram' or unit_string == 'gramme':
        return 1*gram
    elif unit_string == 'hertz':
        return 1*hertz
    elif unit_string == 'joule':
        return 1*joule
    elif unit_string == 'liter' or unit_string == 'litre':
        return 1*liter
    elif unit_string == 'molar':
        return 1*molar
    elif unit_string == 'pascal':
        return 1*pascal
    elif unit_string == 'ohm':
        return 1*ohm
    elif unit_string == 'siemens':
        return 1*siemens
    elif unit_string == 'volt':
        return 1*volt
    elif unit_string == 'watt':
        return 1*watt
    elif unit_string == 'msiemens':
        return 1*msiemens
    elif unit_string == 'ms':
        return 1*msecond
    elif unit_string == 'hz' or unit_string=='Hz':
        return 1*hertz
    elif unit_string == 'mm':
        return 1*mmolar
    elif unit_string == 'cm' or unit_string == 'cmeter' or unit_string == 'cmetre':
        return 1*cmetre
    elif unit_string == 'ns':
        return 1*nsiemens
    elif unit_string == 'mV':
        return 1*mV
    else:
        return False


def visualise_condition_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), 'ok', ms=10)
    plot(ones(Nt), arange(Nt), 'ok', ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], '-k')
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    subplot(122)
    plot(S.i, S.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')


def tranform_input(input, option):
    if option == 1:
        if input.startswith('[') and input.endswith(']'):
            input = eval(input)
        else:
            input = int(input)
    elif option == 2:
        if input.startswith('[') and input.endswith(']'):
            input = input.strip('[]').replace(' ', '').split(',')
    return input


def visualise_direct_connectivity(M, n):
    for i in range(n):
        plot(M.t/ms, M.v[i], label='Neuron '+str(i))
    xlabel('Time (ms)')
    ylabel('v')


def change_eqs(eqs, Str, type):
    if eqs == '':
        return eqs
    equations = eqs.split("@")
    if Str == '':
        eqs = "\n".join(equations)
    else:
        if type == 0:
            equations[0] = equations[0] + ' ' + Str
        else:
            if len(equations) != 1:
                for i in range(1, len(equations)):
                    equations[i] = equations[i] + ' ' + Str
        eqs = "\n".join(equations)
    return eqs


def char2var(variable_name):
    return globals()[variable_name]


def char2attr(M, variable_name):
    return getattr(M, variable_name)


def get_input_values(end_text_boxes):
    input_values = {}
    for group_index, text_boxes in end_text_boxes.items():
        group_values = []
        for text_box in text_boxes:
            if hasattr(text_box, 'value'):
                group_values.append(text_box.value)
            else:
                group_values.append(text_box.children)
        input_values[group_index] = group_values
    return input_values


def var_plot(Mx, varx, My, vary, i):
    string = "plot(char2attr(Mx,varx), char2attr(My,vary))"
    if varx == 't':
        if type(My) != SpikeMonitor:
            string = string.replace(
                "char2attr(My,vary)", "char2attr(My,vary)[i]")
    else:
        if type(Mx) != SpikeMonitor:
            string = string.replace(
                "char2attr(Mx,varx)", "char2attr(Mx,varx)[i]")
        if type(My) != SpikeMonitor:
            string = string.replace(
                "char2attr(My,vary)", "char2attr(My,vary)[i]")
    exec(string)

def var_plot1(Mx, varx, My, vary, i):
    string = "plot(char2attr(Mx,varx), char2attr(My,'i'),'.k')"
    if varx == 't':
        if type(My) != SpikeMonitor:
            string = string.replace(
                "char2attr(My,vary)", "char2attr(My,vary)[i]")
    else:
        if type(Mx) != SpikeMonitor:
            string = string.replace(
                "char2attr(Mx,varx)", "char2attr(Mx,varx)[i]")
        if type(My) != SpikeMonitor:
            string = string.replace(
                "char2attr(My,vary)", "char2attr(My,vary)[i]")
    exec(string)

def var_plot2(Mx, varx, My, vary, i):
    subplot(122)
    duration = 1000*ms
    string = "plot(char2attr(Mx,varx), char2attr(My,vary)/duration)"
    exec(string)

def dashed_line_plot(spikemon):
    for t in spikemon.t:
        axvline(t/second, ls='--', c='C1', lw=3)


def num_1(record):
    if type(record) == int:
        return record - 1
    elif type(record) == list:
        return [num - 1 for num in record]


def classify_input(input_string):
    try:
        float(input_string)
        return tranform_input(input_string, 1)
    except ValueError:
        pass
    try:
        elements = eval(input_string)
        if all(isinstance(elem, (int, float)) for elem in elements):
            return tranform_input(input_string, 1)
        elif all(isinstance(elem, str) for elem in elements):
            return tranform_input(input_string, 2)
        else:
            return 'unknown'
    except:
        pass
    return tranform_input(input_string, 2)


def special_str(str):
    return False


def extract_parameters(str):
    # Process the string to get the parameters and variables
    pattern = r'(\b[A-Za-z_][A-Za-z_0-9]*\b)'
    pattert2 = r'[d][a-z]'
    parameters = re.findall(pattern, str)
    rst = set()
    for p in parameters:
        if str2quantity(p) == False:  # Non-unit
            # to_do:Non-special characters
            if p != 'exp' and p != 'sqrt':
                match = re.match(pattert2, p)
                if match == None:  # Non-differential
                    rst.add(p)
    return rst

def show_synaptic_connectivity_diagram(N,mode,S,ii='',jj=''):
    np.random.seed(0)  # Setting random seeds to maintain reproducibility
    neuron_positions_x = np.random.rand(N)  # Generate random positions between 0 and 1
    neuron_positions_y = np.random.rand(N)  # Generate random positions between 0 and 1
    # Creating drawing objects
    fig = plt.figure(figsize=(8, 6))

    # Plot spatial distribution and labelling of neurons
    for i in range(N):
        plt.scatter(neuron_positions_x[i], neuron_positions_y[i], marker='o', color='darkred')  # Plot neuron positions
        plt.text(neuron_positions_x[i], neuron_positions_y[i]+0.012, str(i+1), ha='center', color='darkred')  # Add neuron labels
    if mode == 0:
        if ii == '':
            for i, j in zip(S.i, S.j):
                weight = S.w[S.i == i][0]
                plt.plot([neuron_positions_x[i], neuron_positions_x[j]], [neuron_positions_y[i], neuron_positions_y[j]], '-b')
                plt.text((neuron_positions_x[i] + neuron_positions_x[j]) / 2,
                (neuron_positions_y[i] + neuron_positions_y[j]) / 2,
                f'{float(weight)}', ha='center', va='center')  # Adding weight tags
        elif jj == '':
            for i, j in zip(S.i, S.j):
                weight = S.w[S.i == i][0]
                plt.plot([neuron_positions_x[i], neuron_positions_x[j]], [neuron_positions_y[i], neuron_positions_y[j]], '-b')
                plt.text((neuron_positions_x[i] + neuron_positions_x[j]) / 2,
                (neuron_positions_y[i] + neuron_positions_y[j]) / 2,
                f'{float(weight)}', ha='center', va='center')  # Adding weight tags
        else:
            if type(ii)==list:
                if type(jj)==list:
                    for iii in ii:
                        for jjj in jj:
                            plt.plot([neuron_positions_x[iii], neuron_positions_x[jjj]], [neuron_positions_y[iii], neuron_positions_y[jjj]], '-b')
                            weight = S.w[S.i == iii][0]
                            plt.text((neuron_positions_x[iii] + neuron_positions_x[jjj]) / 2,
                            (neuron_positions_y[iii] + neuron_positions_y[jjj]) / 2,
                            f'{float(weight)}', ha='center', va='center')  # Adding weight tags
                else:
                    for iii in ii:
                        plt.plot([neuron_positions_x[iii], neuron_positions_x[jj]], [neuron_positions_y[iii], neuron_positions_y[jj]], '-b')
                        weight = S.w[S.i == iii][0]
                        plt.text((neuron_positions_x[iii] + neuron_positions_x[jj]) / 2,
                            (neuron_positions_y[iii] + neuron_positions_y[jj]) / 2,
                            f'{float(weight)}', ha='center', va='center')  # Adding weight tags
            else:
                if type(jj)==list:
                    for jjj in jj:
                        plt.plot([neuron_positions_x[ii], neuron_positions_x[jjj]], [neuron_positions_y[ii], neuron_positions_y[jjj]], '-b')
                        weight = S.w[S.j == jjj][0]
                        plt.text((neuron_positions_x[ii] + neuron_positions_x[jjj]) / 2,
                        (neuron_positions_y[ii] + neuron_positions_y[jjj]) / 2,
                        f'{float(weight)}', ha='center', va='center')  # Adding weight tags
                else:
                    plt.plot([neuron_positions_x[ii], neuron_positions_x[jj]], [neuron_positions_y[ii], neuron_positions_y[jj]], '-b')
                    weight = S.w[0]
                    plt.text((neuron_positions_x[ii] + neuron_positions_x[jj]) / 2,
                        (neuron_positions_y[ii] + neuron_positions_y[jj]) / 2,
                        f'{float(weight)}', ha='center', va='center')  # Adding weight tags

    else:
        for i, j in zip(S.i, S.j):
            weight = S.w[S.i == i][0]
            plt.plot([neuron_positions_x[i], neuron_positions_x[j]], [neuron_positions_y[i], neuron_positions_y[j]], '-b')
            plt.text((neuron_positions_x[i] + neuron_positions_x[j]) / 2,
             (neuron_positions_y[i] + neuron_positions_y[j]) / 2,
             f'{float(weight)}', ha='center', va='center')  # Adding weight tags


In [17]:
#Determine whether to run multiple times, and if so, display the corresponding input box.
def is_multi_change(change):
    if is_multi.value:
        range_widget.layout.display='block'
        num_runs_widget.layout.display='block'
        duration_widget.layout.display='block'
        radio_control_widget.layout.display='block'
        run_time_text.layout.display = 'none'
    else:
        run_time_text.layout.display = 'block'
        interspike_widget.layout.display = 'none'
        range_widget.layout.display='none'
        num_runs_widget.layout.display='none'
        duration_widget.layout.display='none'
        radio_control_widget.layout.display='none'


is_multi = widgets.Checkbox(value=False,description='multiple runs',indent=False,style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
is_multi.observe(is_multi_change,'value')

#Single-run components
run_time = 100*ms
run_time_text = widgets.Text(value='100ms',description='Please enter simulation time:(eg:100ms)(required)',style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
def run_time_input(sender):  # Receive text box input
    global run_time
    run_time = sender.new


run_time_text.observe(run_time_input, names='value')


#Multi-run controls
def radio_change(change):
    interspike_widget.layout.display = 'none'
    if radio_control_widget.value == 3:
        interspike_widget.layout.display = 'block'

range_widget = widgets.Text(value='30ms',description='time range',style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
num_runs_widget = widgets.IntText(value=1, description='number of runs',style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
duration_widget = widgets.Text(value='1second',description='duration per run',style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
radio_control_widget = widgets.RadioButtons(
    options=[('One-time network creation',0),('Multiple network creations',1), ('The time constant effect',2), ('The time constant as a parameter',3)],
    value=0,
    description='Pattern selection',style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px')
)
radio_control_widget.observe(radio_change,'value')
interspike_widget = widgets.Checkbox(value=False, description='interspike Interval',style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
interspike_widget.layout.display = 'none'
range_widget.layout.display ='none'
num_runs_widget.layout.display='none'
duration_widget.layout.display ='none'
radio_control_widget.layout.display ='none'

#simplify info of exception
is_custom_exception = False

def custom_exception_checkbox_callback(change):
    global is_custom_exception
    is_custom_exception = change['new']

custom_exception_checkbox = widgets.Checkbox(
    value=False,
    description='Simplify error messages(When an error occurs, only the error type and the reason will be displayed)',
    disabled=False,
    style={
                                              'description_width': 'initial'}, layout=widgets.Layout(width='700px'),
    indent=False
)
custom_exception_checkbox.observe(
    custom_exception_checkbox_callback, names='value')

#back-end equipment
device_drpbx = widgets.Dropdown(options=[("numpy", 1), ("cython", 2), (
    "weave", 3), ("numba", 4), ("cpp_standalone", 5)], index=0, value=1, label="numpy",description='Please select the backend device:',style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
Device = 'numpy'
device_text = widgets.Text(description='Also can fill in the backend device:',style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
def device_input(sender):  # Receive text box input
    global Device
    Device = sender.new

device_text.observe(device_input, names='value')

def device_chosen(_):  # Receive drop-down box content
    global Device
    Device = device_drpbx.label

device_drpbx.observe(device_chosen, names="value")
global_main_box = widgets.VBox([
    is_multi, run_time_text,
    range_widget, num_runs_widget, duration_widget,
    radio_control_widget, interspike_widget,device_drpbx, device_text,
    custom_exception_checkbox
])

In [18]:
num_groups = 0
parameters = {}


def is_change_change(self):
    index = accordion.selected_index
    accordion.children[index].children[10].value = False
    accordion.children[index].children[9].layout.display = 'none'
    accordion.children[index].children[10].layout.display = 'none'
    if accordion.children[index].children[8].value:
        accordion.children[index].children[9].layout.display = 'block'
        accordion.children[index].children[10].layout.display = 'block'


def on_click(self):
    # if gui != None:
    #     gui.layout.display = 'None'
    #     parameter_values = 'None'
    index = accordion.selected_index
    str = accordion.children[index].children[1].children[0].children[0].value
    rst = extract_parameters(str)
    g, parameter_values = create_gui(rst)
    accordion.children[index].children[1].children[1].children = g.children
    accordion.children[index].children[1].children[1].set_title(0, 'param')
    accordion.children = accordion.children
    parameters[index] = parameter_values


def num_groups_changed(change):
    global num_groups
    num_groups = change.new
    accordion.children = ()
    for i in range(num_groups):
        if i in neurons_text_boxes:
            num_neurons_texts = neurons_text_boxes[i]
        else:
            num_neurons_texts = []
            #Number of neurons input box
            neurons_num_text = widgets.IntText(value=1, description='Please enter the number of neurons (required):', style={
                                               'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            #Differential Equation Input Box
            eqs_text = widgets.Textarea(description='Please enter the differential equation(s) (separated by @, required):', style={
                                        'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            confirm_btn = widgets.Button(description='Confirm')
            confirm_btn.on_click(on_click)
            eqs_text_btn = widgets.HBox([eqs_text, confirm_btn])
            eqs_para = widgets.VBox([eqs_text_btn, widgets.Accordion()])
            #Numerical integration methods
            method_text = widgets.Text(description='Please enter the numerical integration method:(required)', value='exact', style={
                                       'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            #Thresholds
            threshold_text = widgets.Text(description='Please enter the threshold (e.g., v > 0.8, optional):', value='', style={
                                          'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            #Reset
            reset_text = widgets.Text(description='Please enter the reset condition (e.g., v = 0, optional):', value='', style={
                                      'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            #refractory period
            refractory_text = widgets.Text(description='Please enter the refractory period (including units, optional):', value='0*ms', style={
                                           'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            #Recorded pulses
            dashed_line_checkbox = widgets.Checkbox(
                value=False,
                description='record spikes',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='700px'),
                disabled=False,
                indent=False
            )
            #unless refractory
            unless_refractory_checkbox = widgets.Checkbox(
                value=False,
                description='add unless refractory',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='700px'),
                disabled=False,
                indent=False
            )

            is_change_wgt = widgets.Checkbox(value=False, description="Changing variables during runtime:", style={
                                             'description_width': 'initial'},indent=False)
            is_change_wgt.observe(is_change_change, 'value')
            #Variable Rules
            code_input_wgt = widgets.Textarea(value='', description="rules:")
            #shared variable
            is_share_wgt = widgets.Checkbox(value=False, description='Shared variables')
            is_share_wgt.layout.display = 'none'
            code_input_wgt.layout.display = 'none'
            num_neurons_texts.append(neurons_num_text)#0
            num_neurons_texts.append(eqs_para)
            num_neurons_texts.append(method_text)#2
            num_neurons_texts.append(threshold_text)
            num_neurons_texts.append(reset_text)#4
            num_neurons_texts.append(refractory_text)
            num_neurons_texts.append(dashed_line_checkbox)#6
            num_neurons_texts.append(unless_refractory_checkbox)
            num_neurons_texts.append(is_change_wgt)#8
            num_neurons_texts.append(code_input_wgt)
            num_neurons_texts.append(is_share_wgt)#10
            neurons_text_boxes[i] = num_neurons_texts
        accordion.children += (widgets.VBox(num_neurons_texts),)
        accordion.set_title(i, f'Neurons Group {i+1}')


num_groups_text = widgets.IntText(value=0, description='Please enter the number of neuron groups:', style={
                                  'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
neurons_text_boxes = {}
accordion = widgets.Accordion()
num_groups_text.observe(num_groups_changed, names='value')
main_box = widgets.VBox([num_groups_text, accordion])

In [19]:
s_parameters = {}


def s_on_click_wf(self):
    index = s_accordion.selected_index
    str = s_accordion.children[index].children[2].children[0].children[0].value
    rst = extract_parameters(str)
    g, parameter_values = create_gui(rst)
    s_accordion.children[index].children[2].children[1].children = g.children
    s_accordion.children[index].children[2].children[1].set_title(
        0, 'parameters')
    s_parameters[index] = parameter_values


def s_build_chosen(change, s_direct_i_text, s_direct_j_text, s_condition_text, s_p_text):  # Receive drop-down box content
    if change.new == 1:
        s_direct_i_text.layout.display = 'block'
        s_direct_j_text.layout.display = 'block'
        s_condition_text.layout.display = 'none'
        s_p_text.layout.display = 'none'
    elif change.new == 2:
        s_condition_text.layout.display = 'block'
        s_direct_i_text.layout.display = 'none'
        s_direct_j_text.layout.display = 'none'
        s_p_text.layout.display = 'block'


num_s = 0


def num_s_changed(change):
    global num_s
    num_s = change.new
    s_accordion.children = ()
    for i in range(num_s):
        if i in s_neurons_text_boxes:
            s_num_neurons_texts = s_neurons_text_boxes[i]
        else:
            s_num_neurons_texts = []
            s_build_drpbx = widgets.Dropdown(options=[], description='Please select the synaptic connectivity mode:', style={
                                             'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_build_drpbx.options = [("Direct connection", 1), ("Conditional connection", 2)]
            s_direct_i_text = widgets.Text(value='', description='Please specify the neuron(s) to connect to (you can enter an array,starting from 1):', style={
                                           'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_direct_j_text = widgets.Text(value='', description='Please specify the neuron(s) to connect from (you can enter an array,starting from 1):', style={
                                           'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_condition_text = widgets.Text(value='', description='Please specify the connection condition:', style={
                                            'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_p_text = widgets.Text(value='1', description='Please specify the connection probability:', style={
                                    'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_direct_i_text.layout.display = 'block'
            s_direct_j_text.layout.display = 'block'
            s_condition_text.layout.display = 'none'
            s_p_text.layout.display = 'none'
            s_build_drpbx.observe(lambda change: s_build_chosen(
                change, s_direct_i_text, s_direct_j_text, s_condition_text, s_p_text), names='value')
            s_neurons_groups_get_text_begin = widgets.IntText(value=0, description='Please select the presynaptic neuron group(required,starting from 1):', style={
                                                              'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_neurons_groups_get_text_end = widgets.IntText(value=0, description='Please select the postsynaptic neuron group(required,starting from 1):', style={
                                                            'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_eqs_text = widgets.Textarea(value='w:1', description='Please enter the differential equation(s) for the synapse(s) (required)(separated by @):', style={
                                          'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            # Add parameters
            s_eqs_btn = widgets.Button(description='confirm')
            s_eqs_btn.on_click(s_on_click_wf)
            s_eqs_btn = widgets.HBox([s_eqs_text, s_eqs_btn])
            s_eqs_para = widgets.VBox([s_eqs_btn, widgets.Accordion()])
            s_w_text = widgets.Text(description='Please specify the synaptic weight(s)(eg : j * 0.2)(optional):', style={
                                    'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_on_pre_text = widgets.Textarea(value='', description='Please enter the on_pre equation(s)(optional) (separated by @):', style={
                                             'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_on_post_text = widgets.Textarea(value='', description='Please enter the on_post equation(s) (optional)(separated by @):', style={
                                              'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_event_drpbx = widgets.Dropdown(options=[], description='Please select the event-driven mode(optional):', style={
                                             'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_event_drpbx.options = [("event-driven", 1), ("clock-driven", 2)]
            s_delay_text = widgets.Text(description='Please specify the synaptic delay(eg : j * 2 * ms)(optional):', style={
                                        'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_method_text = widgets.Text(description='Please enter the numerical integration method(required):', value='linear', style={
                                         'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            skip_if_invalid_checkbox = widgets.Checkbox(
                value=False,
                description='avoid boundary errors',
                disabled=False,
                indent=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='700px')
            )
            s_diagram_checkbox = widgets.Checkbox(
                value=False,
                description='show synaptic connectivity diagram',
                disabled=False,
                indent=False, style={'description_width': 'initial'}, layout=widgets.Layout(width='700px')
            )
            s_num_neurons_texts.append(s_neurons_groups_get_text_begin)
            s_num_neurons_texts.append(s_neurons_groups_get_text_end)
            s_num_neurons_texts.append(s_eqs_para)
            s_num_neurons_texts.append(s_build_drpbx)
            s_num_neurons_texts.append(s_direct_i_text)
            s_num_neurons_texts.append(s_direct_j_text)
            s_num_neurons_texts.append(s_condition_text)
            s_num_neurons_texts.append(s_p_text)
            s_num_neurons_texts.append(s_on_pre_text)
            s_num_neurons_texts.append(s_on_post_text)
            s_num_neurons_texts.append(s_w_text)
            s_num_neurons_texts.append(s_event_drpbx)
            s_num_neurons_texts.append(s_delay_text)
            s_num_neurons_texts.append(s_method_text)
            s_num_neurons_texts.append(skip_if_invalid_checkbox)
            s_num_neurons_texts.append(s_diagram_checkbox)
            s_neurons_text_boxes[i] = s_num_neurons_texts
        s_accordion.children += (widgets.VBox(s_num_neurons_texts),)
        s_accordion.set_title(i, f'Synapse {i+1}')


s_num_text = widgets.IntText(value=0, description='Please enter the number of synapses:', style={
                             'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
s_neurons_text_boxes = {}
s_accordion = widgets.Accordion()
s_num_text.observe(num_s_changed, names='value')
s_main_box = widgets.VBox([s_num_text, s_accordion])

In [20]:
num_track = 0


def num_track_changed(change):
    global num_track
    num_track = change.new
    track_accordion.children = ()
    for i in range(num_track):
        if i in track_text_boxes:
            track_texts = track_text_boxes[i]
        else:
            track_texts = []
            track_drpbx = widgets.Dropdown(options=[], description='Please select the tracking target type:', style={
                                           'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            track_drpbx.options = [("Neuron", 1), ("Synapse", 2)]
            track_target_get_text = widgets.IntText(value=0, description='Please select the tracking target(starting from 1):', style={
                                                    'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            track_var_text = widgets.Text(description='Please enter the variable(s) you want to track (you can enter an array, required):', value='', style={
                                          'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            track_var_neu_text = widgets.Text(description='Please enter the neuron(s) you want to track (you can enter an array, required):', value='True', style={
                                              'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            track_texts.append(track_drpbx)
            track_texts.append(track_target_get_text)
            track_texts.append(track_var_text)
            track_texts.append(track_var_neu_text)
            track_text_boxes[i] = track_texts
        track_accordion.children += (widgets.VBox(track_texts),)
        track_accordion.set_title(i, f'Track {i+1}')


track_text = widgets.IntText(value=0, description='Please enter the number of track:', style={
                             'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
track_text_boxes = {}
track_accordion = widgets.Accordion()
track_text.observe(num_track_changed, names='value')
track_main_box = widgets.VBox([track_text, track_accordion])

In [21]:
num_visual = 0


def visual_build_chosen(change, s_groups_get_text, text_boxes_layout):  # Receive drop-down box content
    if change.new == 1:
        s_groups_get_text.layout.display = 'none'
        text_boxes_layout.layout.display = 'block'
    elif change.new == 2:
        s_groups_get_text.layout.display = 'block'
        text_boxes_layout.layout.display = 'none'


def num_visual_changed(change):
    global num_visual
    num_visual = change.new
    visual_accordion.children = ()
    for i in range(num_visual):
        if i in visual_text_boxes:
            visual_texts = visual_text_boxes[i]
        else:
            visual_texts = []
            visual_drpbx = widgets.Dropdown(options=[("View the relationship between two variables", 1), ("View the connectivity of neurons", 2)], description='Please select a visualization method:', style={
                                            'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_groups_get_text = widgets.IntText(value=0, description='Please enter the selected synapse(s):', style={
                'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            s_groups_get_text.layout.display = 'none'
            select_drpbx = widgets.Dropdown(options=[("Neuron", 1), ("Synapse", 2)], description='Please select the category:', index=0, value=1, label="Neuron", style={
                'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            neurons_groups_get_text = widgets.IntText(value=0, description='Please enter the selected neuron group/synapse(starting from 1):', style={
                'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            neurons_get_text = widgets.IntText(value=0, description='Please enter the selected neuron(starting from 1):', style={
                'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            visual_x_drpbx = widgets.Dropdown(options=[("Neuron", 1), ("Spike", 2)], description="Please select the section to which variable x belongs:", style={
                'description_width': 'initial'}, layout=widgets.Layout(width='700px'), value=1, label="Neuron")
            var_text_boxy = widgets.Text(description='Please enter the variable y to visualize:', style={
                'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            visual_y_drpbx = widgets.Dropdown(options=[("Neuron", 1), ("Spike", 2)], description="Please select the section to which variable y belongs:", style={
                'description_width': 'initial'}, layout=widgets.Layout(width='700px'), value=1, label="Neuron")
            var_text_boxx = widgets.Text(value='t', description='Please enter the variable x to visualize:', style={
                'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
            dashed_line_checkbox1 = widgets.Checkbox(
                value=False,
                description='show the spike firing time points',
                style={'description_width': 'initial'},
                layout=widgets.Layout(width='700px'),
                disabled=False,
                indent=False
            )
            text_boxes_layout = widgets.VBox([select_drpbx, neurons_groups_get_text, neurons_get_text,
                                              visual_x_drpbx, var_text_boxx, visual_y_drpbx, var_text_boxy, dashed_line_checkbox1])
            text_boxes_layout.layout.display = 'block'
            visual_drpbx.observe(lambda change: visual_build_chosen(
                change, s_groups_get_text, text_boxes_layout), names='value')
            visual_texts.append(visual_drpbx)
            visual_texts.append(text_boxes_layout)
            visual_texts.append(s_groups_get_text)
            visual_text_boxes[i] = visual_texts

        visual_accordion.children += (widgets.VBox(visual_texts),)
        visual_accordion.set_title(i, f'Visualization {i+1}')


visual_text = widgets.IntText(value=0, description='Please enter the number of visualizations:', style={
                              'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
visual_text_boxes = {}
visual_accordion = widgets.Accordion()
visual_text.observe(num_visual_changed, names='value')
visual_main_box = widgets.VBox([visual_text, visual_accordion])

In [22]:
def main():
    global num_groups, run_time, eqs
    global s_delay, s_w, G, s_neurons_text_boxes, neurons_text_boxes, track_text_boxes
    input_values = get_input_values(neurons_text_boxes)
    s_input_values = get_input_values(s_neurons_text_boxes)
    track_input_values = get_input_values(track_text_boxes)
    visual_input_values = get_input_values(visual_text_boxes)
    run_time = match_digital(run_time)
    taupre = taupost = 20*ms
    wmax = 0.01
    Apre = 0.01
    Apost = -Apre*taupre/taupost*1.05
    try:

        prefs.codegen.target = Device
        start_scope()
        for i in range(num_groups):
            if input_values[i][7]:
                eqs = "'''" +change_eqs(input_values[i][1][0].children[0].value, '(unless refractory)',0) + "'''"
            else:
                eqs = "'''" + change_eqs(input_values[i][1][0].children[0].value,'',0) + "'''"
            if input_values[i][10]:
                v = input_values[i][9][0]
                e = re.findall(rf"{v} : \b[A-Za-z]*\b", eqs)
                eqs = re.sub(rf"{v} : \b[A-Za-z]*\b", e[0] + ' (shared)', eqs)
            if input_values[i][3] == '':
                exec(
                    f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, method='{input_values[i][2]}')")
            else:
                matches = re.findall(r"(\d+)(\D+)", input_values[i][5])
                if matches:
                    value = float(matches[0][0])
                    unit = matches[0][1]
                exec(
                    f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, threshold='{input_values[i][3]}', reset='{input_values[i][4]}', refractory=input_values[i][5], method='{input_values[i][2]}')")

            for p in parameters[i]:
                if ('d'+p in eqs or re.search(rf'^{p} : ',eqs,re.MULTILINE) != None)  and parameters[i][p].value != "None":
                    exec(f"G{i}.{p} = {parameters[i][p].value}")
                else:
                    if parameters[i][p].value != "None":
                        exec(f"{p} = {parameters[i][p].value}",globals())
            if input_values[i][6]:
                exec(f"spikemon{i} = SpikeMonitor(G{i})")
            if input_values[i][8]:
                #Changing variables on the fly
                exec(f"G{i}.run_regularly(input_values[i][9],dt=10*ms)")


        for i in range(num_s):
            event = ''
            if s_input_values[i][11] == 1:
                event = '(event-driven)'
            elif s_input_values[i][11] == 2:
                event = '(clock-driven)'
            s_eqs = change_eqs(
                s_input_values[i][2][0].children[0].value, event, 1)
            s_eqs = "'''" + s_eqs + "'''"
            on_pre = change_eqs(s_input_values[i][8], '',0)
            on_pre = "'''" + on_pre + "'''"
            on_post = change_eqs(s_input_values[i][9], '',0)
            on_post = "'''" + on_post + "'''"
            exec(
                f"S{i} = Synapses(G{s_input_values[i][0]-1}, G{s_input_values[i][1]-1}, {s_eqs}, on_pre={on_pre},on_post={on_post},method='{s_input_values[i][13]}')")
            for p in s_parameters[i]:
                if ('d'+p in s_eqs or re.search(rf'^{p} : ',s_eqs,re.MULTILINE) != None)  and s_parameters[i][p].value != "None":
                    exec(f"G{i}.{p} = {s_parameters[i][p].value}")
                else:
                    if s_parameters[i][p].value != "None":
                        exec(f"{p} = {s_parameters[i][p].value}",globals())
            ii = ''
            jj = ''
            if s_input_values[i][3] == 1:
                if s_input_values[i][4] == '':
                    jj = tranform_input(s_input_values[i][5], 2)
                    exec(
                        f"S{i}.connect(j='{jj}',skip_if_invalid={s_input_values[i][14]})")
                elif s_input_values[i][5] == '':
                    ii = tranform_input(s_input_values[i][4], 2)
                    exec(
                        f"S{i}.connect(i='{ii}',skip_if_invalid={s_input_values[i][14]})")
                else:
                    ii = tranform_input(s_input_values[i][4], 1)
                    ii = num_1(ii)
                    jj = tranform_input(s_input_values[i][5], 1)
                    jj = num_1(jj)
                    exec(
                        f"S{i}.connect(i={ii},j={jj},skip_if_invalid={s_input_values[i][14]})")

            elif s_input_values[i][3] == 2:
                exec(
                    f"S{i}.connect(condition='{s_input_values[i][6]}',p={float(s_input_values[i][7])},skip_if_invalid={s_input_values[i][14]})")
            if s_input_values[i][10] != '':
                exec(f"S{i}.w='{s_input_values[i][10]}'")
            if s_input_values[i][12] != '':
                exec(f"S{i}.delay='{s_input_values[i][12]}'")
            if s_input_values[i][15] == True:
                if s_input_values[i][3] == 1:
                    if ii=='':
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},jj={jj})")
                    elif jj=='':
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii})")
                    else:
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii},jj={jj})")

                else:
                    exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],1,S{i})")

        for i in range(num_track):
            varr = tranform_input(track_input_values[i][2], 2)
            if type(varr) == list:
                track_string = f"statemon{i} = StateMonitor(G{track_input_values[i][1]-1}, {tranform_input(track_input_values[i][2], 2)}, record=True)"
            else:
                track_string = f"statemon{i} = StateMonitor(G{track_input_values[i][1]-1}, '{tranform_input(track_input_values[i][2], 2)}', record=True)"
            if track_input_values[i][0] == 2:
                track_string = track_string.replace('G', 'S')
            if track_input_values[i][3].lower() == "false":
                track_string = track_string.replace('True', 'False')
                exec(
                    f"statemon{i} = StateMonitor(G{i}, '{tranform_input(input_values[i][6], 2)}', record=False)")
            elif track_input_values[i][3].lower() == "true":
                pass
            else:
                record = tranform_input(track_input_values[i][3], 1)
                record = num_1(record)
                record = str(record)
                track_string = track_string.replace('True', record)
            exec(track_string)
        run(run_time)
        for i in range(num_visual):
            if visual_input_values[i][0] == 1:
                string = f"var_plot(statemonx{int(visual_input_values[i][1][1].value)-1},visual_input_values[i][1][4].value,statemony{int(visual_input_values[i][1][1].value)-1},visual_input_values[i][1][6].value,int(visual_input_values[i][1][2].value)-1)"
                if visual_input_values[i][1][3].value == 1:
                    string = string.replace('statemonx', 'statemon')
                elif visual_input_values[i][1][3].value == 2:
                    string = string.replace('statemonx', 'spikemon')
                if visual_input_values[i][1][5].value == 1:
                    string = string.replace('statemony', 'statemon')
                elif visual_input_values[i][1][5].value == 2:
                    string = string.replace('statemony', 'spikemon')
                if visual_input_values[i][1][6].value == 'i':
                    string = string.replace('var_plot', 'var_plot1')
                elif visual_input_values[i][1][6].value == 'count':
                    string = string.replace('var_plot', 'var_plot2')
                    string = string.replace('statemon', 'G')
                exec(string)
                if visual_input_values[i][1][7].value == True:
                    exec(
                        f"dashed_line_plot(spikemon{int(visual_input_values[i][1][1].value)-1})")
            elif visual_input_values[i][0] == 2:
                exec(
                    f"visualise_condition_connectivity(S{int(visual_input_values[i][2])-1})")

    except Exception as e:
        if is_custom_exception:
            error_type = type(e).__name__
            print(f"{error_type}: {str(e)}")
        else:
            raise


In [23]:
def mode0():    #Single network creation
    global num_groups, run_time, eqs
    global s_delay, s_w, G, s_neurons_text_boxes, neurons_text_boxes, track_text_boxes
    input_values = get_input_values(neurons_text_boxes)
    s_input_values = get_input_values(s_neurons_text_boxes)
    track_input_values = get_input_values(track_text_boxes)
    run_time = match_digital(run_time)
    time_range = range_widget.value
    num_runs = num_runs_widget.value
    duration_str = duration_widget.value
    tau_range = linspace(1,match_digital2(time_range)[0],num_runs)*match_digital2(time_range)[1]
    duration = match_digital(duration_str)

    try:
        prefs.codegen.target = Device
        start_scope()
        taupre = taupost = 20*ms
        wmax = 0.01
        Apre = 0.01
        Apost = -Apre*taupre/taupost*1.05
        for i in range(num_groups):
            if input_values[i][7]:
                eqs = "'''" +change_eqs(input_values[i][1][0].children[0].value, '(unless refractory)',0) + "'''"
            else:
                eqs = "'''" + change_eqs(input_values[i][1][0].children[0].value,'',0) + "'''"
            # Shared variables
            if input_values[i][10]:
                v = input_values[i][9][0]
                e = re.findall(rf"{v} : \b[A-Za-z]*\b", eqs)
                eqs = re.sub(rf"{v} : \b[A-Za-z]*\b", e[0] + ' (shared)', eqs)
            if input_values[i][3] == '':
                exec(
                    f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, method='{input_values[i][2]}')",globals())

            else:
                matches = re.findall(r"(\d+)(\D+)", input_values[i][5])
                if matches:
                    value = float(matches[0][0])
                    unit = matches[0][1]
                exec(
                    f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, threshold='{input_values[i][3]}', reset='{input_values[i][4]}', refractory={value}*{unit}, method='{input_values[i][2]}')",globals())
            for p in parameters[i]:
                if ('d'+p in eqs or re.search(rf'^{p} : ',eqs,re.MULTILINE) != None)  and parameters[i][p].value != "None":
                    exec(f"G{i}.{p} = {parameters[i][p].value}")
                else:
                    if parameters[i][p].value != "None":
                        exec(f"{p} = {parameters[i][p].value}",globals())
            if input_values[i][6]:
                exec(f"spikemon{i} = SpikeMonitor(G{i})",globals())
        #synapse establishment
        for i in range(num_s):
            event = ''
            if s_input_values[i][11] == 1:
                event = '(event-driven)'
            elif s_input_values[i][11] == 2:
                event = '(clock-driven)'
            s_eqs = change_eqs(
                s_input_values[i][2][0].children[0].value, event, 1)
            s_eqs = "'''" + s_eqs + "'''"
            on_pre = change_eqs(s_input_values[i][8], '',0)
            on_pre = "'''" + on_pre + "'''"
            on_post = change_eqs(s_input_values[i][9], '',0)
            on_post = "'''" + on_post + "'''"
            exec(
                f"S{i} = Synapses(G{s_input_values[i][0]-1}, G{s_input_values[i][1]-1}, {s_eqs}, on_pre={on_pre},on_post={on_post},method='{s_input_values[i][13]}')",globals())
            for p in s_parameters[i]:
                if ('d'+p in s_eqs or re.search(rf'^{p} : ',s_eqs,re.MULTILINE) != None)  and s_parameters[i][p].value != "None":
                    exec(f"G{i}.{p} = {s_parameters[i][p].value}")
                else:
                    if s_parameters[i][p].value != "None":
                        exec(f"{p} = {s_parameters[i][p].value}",globals())
            ii = ''
            jj = ''
            if s_input_values[i][3] == 1:

                if s_input_values[i][4] == '':
                    jj = tranform_input(s_input_values[i][5], 2)
                    exec(
                        f"S{i}.connect(j='{jj}',skip_if_invalid={s_input_values[i][14]})")
                elif s_input_values[i][5] == '':
                    ii = tranform_input(s_input_values[i][4], 2)
                    exec(
                        f"S{i}.connect(i='{ii}',skip_if_invalid={s_input_values[i][14]})")
                else:
                    ii = tranform_input(s_input_values[i][4], 1)
                    ii = num_1(ii)
                    jj = tranform_input(s_input_values[i][5], 1)
                    jj = num_1(jj)
                    exec(
                        f"S{i}.connect(i={ii},j={jj},skip_if_invalid={s_input_values[i][14]})")

            elif s_input_values[i][3] == 2:
                exec(
                    f"S{i}.connect(condition='{s_input_values[i][6]}',p={float(s_input_values[i][7])},skip_if_invalid={s_input_values[i][14]})")
            if s_input_values[i][10] != '':
                exec(f"S{i}.w='{s_input_values[i][10]}'")
            if s_input_values[i][12] != '':
                exec(f"S{i}.delay='{s_input_values[i][12]}'")
            if s_input_values[i][15] == True:
                if s_input_values[i][3] == 1:
                    if ii=='':
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},jj={jj})")
                    elif jj=='':
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii})")
                    else:
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii},jj={jj})")

                else:
                    exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],1,S{i})")

        change_dictionary = {}
        for i in range(num_track):
            track_string = f"statemon{i} = StateMonitor(G{track_input_values[i][1]-1}, '{tranform_input(track_input_values[i][2], 2)}', record=True)"
            if track_input_values[i][0] == 2:
                track_string = track_string.replace('G', 'S')
                change_dictionary[f'S{track_input_values[i][1]-1}'] = i
            else:
                change_dictionary[f'G{track_input_values[i][1]-1}'] = i
            if track_input_values[i][3].lower() == "false":
                track_string = track_string.replace('True', 'False')
                exec(
                    f"statemon{i} = StateMonitor(G{i}, '{tranform_input(input_values[i][6], 2)}', record=False)")
            elif track_input_values[i][3].lower() == "true":
                pass
            else:
                record = tranform_input(track_input_values[i][3], 1)
                record = num_1(record)
                record = str(record)
                track_string = track_string.replace('True', record)
            exec(track_string)
        store()
        output_rates = []
        for tau in tau_range:
            restore()
            run(duration)
            output_rates.append(spikemon1.num_spikes/second)

        plot(tau_range/ms, output_rates)
        xlabel(r'$\tau$ (ms)')
        ylabel('Firing rate (sp/s)');
    except Exception as e:
        if is_custom_exception:
            error_type = type(e).__name__
            print(f"{error_type}: {str(e)}")
        else:
            raise

In [24]:
def mode1():  # Multiple mesh creation
    global num_groups, run_time, eqs
    global s_delay, s_w, G, s_neurons_text_boxes, neurons_text_boxes, track_text_boxes
    input_values = get_input_values(neurons_text_boxes)
    s_input_values = get_input_values(s_neurons_text_boxes)
    track_input_values = get_input_values(track_text_boxes)
    time_range = range_widget.value
    num_runs = num_runs_widget.value
    duration_str = duration_widget.value
    tau_range = linspace(1, match_digital2(time_range)[
                         0], num_runs)*match_digital2(time_range)[1]
    duration = match_digital(duration_str)

    try:
        prefs.codegen.target = Device
        start_scope()
        taupre = taupost = 20*ms
        wmax = 0.01
        Apre = 0.01
        Apost = -Apre*taupre/taupost*1.05
        output_rates = []
        for tau in tau_range:
            for i in range(num_groups):
                if input_values[i][7]:
                    eqs = "'''" +change_eqs(input_values[i][1][0].children[0].value, '(unless refractory)',0) + "'''"
                else:
                    eqs = "'''" + change_eqs(input_values[i][1][0].children[0].value,'',0) + "'''"
                # Shared variables
                if input_values[i][10]:
                    v = input_values[i][9][0]
                    e = re.findall(rf"{v} : \b[A-Za-z]*\b", eqs)
                    eqs = re.sub(rf"{v} : \b[A-Za-z]*\b", e[0] + ' (shared)', eqs)
                if input_values[i][3] == '':
                    exec(
                        f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, method='{input_values[i][2]}')",globals())

                else:
                    matches = re.findall(r"(\d+)(\D+)", input_values[i][5])
                    if matches:
                        value = float(matches[0][0])
                        unit = matches[0][1]
                    exec(
                        f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, threshold='{input_values[i][3]}', reset='{input_values[i][4]}', refractory={value}*{unit}, method='{input_values[i][2]}')",globals())
                for p in parameters[i]:
                    if ('d'+p in eqs or re.search(rf'^{p} : ',eqs,re.MULTILINE) != None)  and parameters[i][p].value != "None":
                        exec(f"G{i}.{p} = {parameters[i][p].value}")
                    else:
                        if parameters[i][p].value != "None":
                            exec(f"{p} = {parameters[i][p].value}",globals())
                if input_values[i][6]:
                    exec(f"spikemon{i} = SpikeMonitor(G{i})",globals())
            #synapse establishment
            for i in range(num_s):
                event = ''
                if s_input_values[i][11] == 1:
                    event = '(event-driven)'
                elif s_input_values[i][11] == 2:
                    event = '(clock-driven)'
                s_eqs = change_eqs(
                    s_input_values[i][2][0].children[0].value, event,1)
                s_eqs = "'''" + s_eqs + "'''"
                on_pre = change_eqs(s_input_values[i][8],'', 0)
                on_pre = "'''" + on_pre + "'''"
                on_post = change_eqs(s_input_values[i][9],'', 0)
                on_post = "'''" + on_post + "'''"
                exec(
                    f"S{i} = Synapses(G{s_input_values[i][0]-1}, G{s_input_values[i][1]-1}, {s_eqs}, on_pre={on_pre},on_post={on_post},method='{s_input_values[i][13]}')",globals())
                for p in s_parameters[i]:
                    if ('d'+p in s_eqs or re.search(rf'^{p} : ',s_eqs,re.MULTILINE) != None)  and s_parameters[i][p].value != "None":
                        exec(f"G{i}.{p} = {s_parameters[i][p].value}")
                    else:
                        if s_parameters[i][p].value != "None":
                            exec(f"{p} = {s_parameters[i][p].value}",globals())
                ii = ''
                jj = ''
                if s_input_values[i][3] == 1:
                    if s_input_values[i][4] == '':
                        jj = tranform_input(s_input_values[i][5], 2)
                        exec(
                            f"S{i}.connect(j='{jj}',skip_if_invalid={s_input_values[i][14]})")
                    elif s_input_values[i][5] == '':
                        ii = tranform_input(s_input_values[i][4], 2)
                        exec(
                            f"S{i}.connect(i='{ii}',skip_if_invalid={s_input_values[i][14]})")
                    else:
                        ii = tranform_input(s_input_values[i][4], 1)
                        ii = num_1(ii)
                        jj = tranform_input(s_input_values[i][5], 1)
                        jj = num_1(jj)
                        exec(
                            f"S{i}.connect(i={ii},j={jj},skip_if_invalid={s_input_values[i][14]})")
                elif s_input_values[i][3] == 2:
                    exec(
                        f"S{i}.connect(condition='{s_input_values[i][6]}',p={float(s_input_values[i][7])},skip_if_invalid={s_input_values[i][14]})")
                if s_input_values[i][10] != '':
                    exec(f"S{i}.w='{s_input_values[i][10]}'")
                if s_input_values[i][12] != '':
                    exec(f"S{i}.delay='{s_input_values[i][12]}'")
                if s_input_values[i][15] == True:
                    if s_input_values[i][3] == 1:
                        if ii=='':
                            exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},jj={jj})")
                        elif jj=='':
                            exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii})")
                        else:
                            exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii},jj={jj})")

                    else:
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],1,S{i})")
            run(duration)
            output_rates.append(spikemon1.num_spikes/second)
            plot(tau_range/ms, output_rates)
            xlabel(r'$\tau$ (ms)')
            ylabel('Firing rate (sp/s)')
    except Exception as e:
        if is_custom_exception:
            error_type = type(e).__name__
            print(f"{error_type}: {str(e)}")
        else:
            raise


In [25]:
def mode2(): #Time constant effects
    global num_groups, run_time, eqs,net,spike_i,spike_t,SGG
    global s_delay, s_w, G, s_neurons_text_boxes, neurons_text_boxes, track_text_boxes
    input_values = get_input_values(neurons_text_boxes)
    s_input_values = get_input_values(s_neurons_text_boxes)
    track_input_values = get_input_values(track_text_boxes)
    time_range = range_widget.value
    num_runs = num_runs_widget.value
    duration_str = duration_widget.value
    tau_range = linspace(1, match_digital2(time_range)[
                         0], num_runs)*match_digital2(time_range)[1]
    duration = match_digital(duration_str)

    try:
        prefs.codegen.target = Device
        start_scope()
        taupre = taupost = 20*ms
        wmax = 0.01
        Apre = 0.01
        Apost = -Apre*taupre/taupost*1.05
        output_rates = []
        for i in range(num_groups):
            if input_values[i][7]:
                eqs = "'''" +change_eqs(input_values[i][1][0].children[0].value, '(unless refractory)',0) + "'''"
            else:
                eqs = "'''" + change_eqs(input_values[i][1][0].children[0].value,'',0) + "'''"
            # shared variable
            if input_values[i][10]:
                v = input_values[0]
                e = re.findall(rf"{v} : \b[A-Za-z]*\b", eqs)
                eqs = re.sub(rf"{v} : \b[A-Za-z]*\b", e[0] + ' (shared)', eqs)
            if input_values[i][3] == '':
                exec(
                    f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, method='{input_values[i][2]}')",globals())
            else:
                matches = re.findall(r"(\d+)(\D+)", input_values[i][5])
                if matches:
                    value = float(matches[0][0])
                    unit = matches[0][1]
                exec(
                    f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, threshold='{input_values[i][3]}', reset='{input_values[i][4]}', refractory={value}*{unit}, method='{input_values[i][2]}')",globals())
            for p in parameters[i]:
                if ('d'+p in eqs or re.search(rf'^{p} : ',eqs,re.MULTILINE) != None)  and parameters[i][p].value != "None":
                    exec(f"G{i}.{p} = {parameters[i][p].value}")
                else:
                    if parameters[i][p].value != "None":
                        exec(f"{p} = {parameters[i][p].value}",globals())
            if i == 0:
                MG0 = SpikeMonitor(G0)
                net = Network(G0,MG0)
                net.run(duration)
                spike_i = MG0.i
                spike_t = MG0.t
                SGG = SpikeGeneratorGroup(input_values[i][0],spike_i,spike_t)
            if input_values[i][6]:
                exec(f"spikemon{i} = SpikeMonitor(G{i})",globals())
        #synapse establishment
        for i in range(num_s):
            event = ''
            if s_input_values[i][11] == 1:
                event = '(event-driven)'
            elif s_input_values[i][11] == 2:
                event = '(clock-driven)'
            s_eqs = change_eqs(
                s_input_values[i][2][0].children[0].value, event,1)
            s_eqs = "'''" + s_eqs + "'''"
            on_pre = change_eqs(s_input_values[i][8], '',0)
            on_pre = "'''" + on_pre + "'''"
            on_post = change_eqs(s_input_values[i][9], '',0)
            on_post = "'''" + on_post + "'''"
            exec(
                f"S{i} = Synapses(SGG, G1, {s_eqs}, on_pre={on_pre},on_post={on_post},method='{s_input_values[i][13]}')",globals())
            for p in s_parameters[i]:
                if ('d'+p in s_eqs or re.search(rf'^{p} : ',s_eqs,re.MULTILINE) != None)  and s_parameters[i][p].value != "None":
                    exec(f"G{i}.{p} = {s_parameters[i][p].value}")
                else:
                    if s_parameters[i][p].value != "None":
                        exec(f"{p} = {s_parameters[i][p].value}",globals())
            ii = ''
            jj = ''
            if s_input_values[i][3] == 1:

                if s_input_values[i][4] == '':
                    jj = tranform_input(s_input_values[i][5], 2)
                    exec(
                        f"S{i}.connect(j='{jj}',skip_if_invalid={s_input_values[i][14]})")
                elif s_input_values[i][5] == '':
                    ii = tranform_input(s_input_values[i][4], 2)
                    exec(
                        f"S{i}.connect(i='{ii}',skip_if_invalid={s_input_values[i][14]})")
                else:
                    ii = tranform_input(s_input_values[i][4], 1)
                    ii = num_1(ii)
                    jj = tranform_input(s_input_values[i][5], 1)
                    jj = num_1(jj)
                    exec(
                        f"S{i}.connect(i={ii},j={jj},skip_if_invalid={s_input_values[i][14]})")

            elif s_input_values[i][3] == 2:
                exec(
                    f"S{i}.connect(condition='{s_input_values[i][6]}',p={float(s_input_values[i][7])},skip_if_invalid={s_input_values[i][14]})")
            if s_input_values[i][10] != '':
                exec(f"S{i}.w='{s_input_values[i][10]}'")
            if s_input_values[i][12] != '':
                exec(f"S{i}.delay='{s_input_values[i][12]}'")
            if s_input_values[i][15] == True:
                if s_input_values[i][3] == 1:
                    if ii=='':
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},jj={jj})")
                    elif jj=='':
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii})")
                    else:
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii},jj={jj})")

                else:
                    exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],1,S{i})")
        net = Network(SGG,G1,S0,spikemon1)
        net.store()
        output_rates = []
        for tau in tau_range:
            net.restore()
            net.run(duration)
            output_rates.append(spikemon1.num_spikes/second)
        plot(tau_range/ms, output_rates)
        xlabel(r'$\tau$ (ms)')
        ylabel('Firing rate (sp/s)');
    except Exception as e:
        if is_custom_exception:
            error_type = type(e).__name__
            print(f"{error_type}: {str(e)}")
        else:
            raise

In [26]:
def mode3():#Time constant as a parameter
    global num_groups, run_time, eqs,tau_range
    global s_delay, s_w, G, s_neurons_text_boxes, neurons_text_boxes, track_text_boxes
    input_values = get_input_values(neurons_text_boxes)
    s_input_values = get_input_values(s_neurons_text_boxes)
    track_input_values = get_input_values(track_text_boxes)
    run_time = match_digital(run_time)
    time_range = range_widget.value
    num_runs = num_runs_widget.value
    duration_str = duration_widget.value
    tau_range = linspace(1,match_digital2(time_range)[0],num_runs)*match_digital2(time_range)[1]
    num_tau = len(tau_range)
    duration = match_digital(duration_str)


    try:
        prefs.codegen.target = Device
        start_scope()
        taupre = taupost = 20*ms
        wmax = 0.01
        Apre = 0.01
        Apost = -Apre*taupre/taupost*1.05
        for i in range(num_groups):
            if input_values[i][7]:
                eqs = "'''" +change_eqs(input_values[i][1][0].children[0].value, '(unless refractory)',0) + "'''"
            else:
                eqs = "'''" + change_eqs(input_values[i][1][0].children[0].value,'',0) + "'''"
            # shared variable
            if input_values[i][10]:
                v = input_values[i][9][0]
                e = re.findall(rf"{v} : \b[A-Za-z]*\b", eqs)
                eqs = re.sub(rf"{v} : \b[A-Za-z]*\b", e[0] + ' (shared)', eqs)
            if input_values[i][3] == '':
                exec(
                    f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, method='{input_values[i][2]}')",globals())

            else:
                matches = re.findall(r"(\d+)(\D+)", input_values[i][5])
                if matches:
                    value = float(matches[0][0])
                    unit = matches[0][1]
                exec(
                    f"G{i} = NeuronGroup({input_values[i][0]}, {eqs}, threshold='{input_values[i][3]}', reset='{input_values[i][4]}', refractory={value}*{unit}, method='{input_values[i][2]}')",globals())
            for p in parameters[i]:
                if ('d'+p in eqs or re.search(rf'^{p} : ',eqs,re.MULTILINE) != None)  and parameters[i][p].value != "None":
                    exec(f"G{i}.{p} = {parameters[i][p].value}")
                else:
                    if parameters[i][p].value != "None":
                        exec(f"{p} = {parameters[i][p].value}",globals())
            if input_values[i][6]:
                exec(f"spikemon{i} = SpikeMonitor(G{i})",globals())
        #Synapse creation
        for i in range(num_s):
            event = ''
            if s_input_values[i][11] == 1:
                event = '(event-driven)'
            elif s_input_values[i][11] == 2:
                event = '(clock-driven)'
            s_eqs = change_eqs(
                s_input_values[i][2][0].children[0].value, event, 1)
            s_eqs = "'''" + s_eqs + "'''"
            on_pre = change_eqs(s_input_values[i][8], '',0)
            on_pre = "'''" + on_pre + "'''"
            on_post = change_eqs(s_input_values[i][9], '',0)
            on_post = "'''" + on_post + "'''"
            exec(
                f"S{i} = Synapses(G{s_input_values[i][0]-1}, G{s_input_values[i][1]-1}, {s_eqs}, on_pre={on_pre},on_post={on_post},method='{s_input_values[i][13]}')",globals())
            for p in s_parameters[i]:
                if ('d'+p in s_eqs or re.search(rf'^{p} : ',s_eqs,re.MULTILINE) != None)  and s_parameters[i][p].value != "None":
                    exec(f"G{i}.{p} = {s_parameters[i][p].value}")
                else:
                    if s_parameters[i][p].value != "None":
                        exec(f"{p} = {s_parameters[i][p].value}",globals())
            ii = ''
            jj = ''
            if s_input_values[i][3] == 1:

                if s_input_values[i][4] == '':
                    jj = tranform_input(s_input_values[i][5], 2)
                    exec(
                        f"S{i}.connect(j='{jj}',skip_if_invalid={s_input_values[i][14]})")
                elif s_input_values[i][5] == '':
                    ii = tranform_input(s_input_values[i][4], 2)
                    exec(
                        f"S{i}.connect(i='{ii}',skip_if_invalid={s_input_values[i][14]})")
                else:
                    ii = tranform_input(s_input_values[i][4], 1)
                    ii = num_1(ii)
                    jj = tranform_input(s_input_values[i][5], 1)
                    jj = num_1(jj)
                    exec(
                        f"S{i}.connect(i={ii},j={jj},skip_if_invalid={s_input_values[i][14]})")

            elif s_input_values[i][3] == 2:
                exec(
                    f"S{i}.connect(condition='{s_input_values[i][6]}',p={float(s_input_values[i][7])},skip_if_invalid={s_input_values[i][14]})")
            if s_input_values[i][10] != '':
                exec(f"S{i}.w='{s_input_values[i][10]}'")
            if s_input_values[i][12] != '':
                exec(f"S{i}.delay='{s_input_values[i][12]}'")
            if s_input_values[i][15] == True:
                if s_input_values[i][3] == 1:
                    if ii=='':
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},jj={jj})")
                    elif jj=='':
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii})")
                    else:
                        exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],0,S{i},ii={ii},jj={jj})")

                else:
                    exec(f"show_synaptic_connectivity_diagram(input_values[{s_input_values[i][0]-1}][0],1,S{i})")

        run(duration)
        output_rates = spikemon1.count/second
        plot(tau_range/ms, output_rates)
        xlabel(r'$\tau$ (ms)')
        ylabel('Firing rate (sp/s)');
        if interspike_widget.value:
            figure()
            trains = spikemon1.spike_trains()
            isi_mu = full(num_tau, nan)*second
            isi_std = full(num_tau, nan)*second
            for idx in range(num_tau):
                train = diff(trains[idx])
                if len(train)>1:
                    isi_mu[idx] = mean(train)
                    isi_std[idx] = std(train)
            errorbar(tau_range/ms, isi_mu/ms, yerr=isi_std/ms)
            xlabel(r'$\tau$ (ms)')
            ylabel('Interspike interval (ms)');
    except Exception as e:
        if is_custom_exception:
            error_type = type(e).__name__
            print(f"{error_type}: {str(e)}")
        else:
            raise

In [28]:
# @title
# Creating buttons and output areas
global_button = widgets.Button(description="Global", layout=widgets.Layout(flex='1'))
neuron_button = widgets.Button(description="Neuron", layout=widgets.Layout(flex='1'))
synapse_button = widgets.Button(description="Synapse", layout=widgets.Layout(flex='1'))
track_button = widgets.Button(description="Track", layout=widgets.Layout(flex='1'))
visualization_button = widgets.Button(description="Visualization", layout=widgets.Layout(flex='1'))
output_area = widgets.Output()

# Place the components below each function in VBox

Global = widgets.VBox([global_main_box])
Neuron = widgets.VBox([main_box])
Synapse = widgets.VBox([s_main_box])
Track = widgets.VBox([track_main_box])
Visualization = widgets.VBox([visual_main_box])


# Button click event handler
def on_button_clicked(b):
    output_area.clear_output()  # Clear the output area
    with output_area:
        clear_output()
        if b.description == "Global":
            display(Global)
        elif b.description == "Neuron":
            display(Neuron)
        elif b.description == "Synapse":
            display(Synapse)
        elif b.description == "Track":
            display(Track)
        elif b.description == "Visualization":
            display(Visualization)

# Display the buttons and output areas and set the horizontal alignment
buttons = [global_button,neuron_button,synapse_button,track_button,visualization_button]
button_layout = widgets.HBox(buttons)
for button in buttons:
    button.on_click(on_button_clicked)

start_button = widgets.Button(description="Start",button_style='danger', style={
                                               'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
def on_start_button_clicked(b):
    # If run multiple times
    if is_multi.value:
        #mode0
        if radio_control_widget.value == 0:
            mode0()
        elif radio_control_widget.value == 1:
            mode1()
        elif radio_control_widget.value == 2:
            mode2()
        elif radio_control_widget.value == 3:
            mode3()
    else:
        main()

start_button.on_click(on_start_button_clicked)
display(button_layout)
display(output_area)


# Define the code to be re-run
clear_button = widgets.Button(description='Clear', button_style='danger', style={
                                               'description_width': 'initial'}, layout=widgets.Layout(width='700px'))
def clear_and_rerun(_):
    clear_output(wait=False)
    display(button_layout)
    display(output_area)
    display(Button_layout)
clear_button.on_click(clear_and_rerun)
Button_layout = widgets.HBox([start_button,clear_button])
display(Button_layout)

Output()